In [1]:
from __future__ import division

seed_value= 0

import os
os.environ['PYTHONHASHSEED']=str(seed_value)
import random
random.seed(seed_value)
import numpy as np
np.random.seed(seed_value)
import tensorflow as tf
tf.set_random_seed(seed_value)
from keras import backend as K
import os
import keras
import pickle
import os.path
import matplotlib.pyplot as plt
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.callbacks import LambdaCallback
from keras.callbacks import TensorBoard
from sklearn.preprocessing import LabelBinarizer
from keras import initializers


# In[2]:


from keras import backend as K
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 200


# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()


# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
K.set_image_dim_ordering('tf')

K.image_dim_ordering(), x_train.shape


# In[3]:



def build_model(setseed):
    """
    Builds test Keras model for Alexnet CIFAR-10
    :param loss (str): Type of loss - must be one of Keras accepted keras losses
    :return: Keras dense model of predefined structure
    """
    input = Input(shape=[32, 32,3])
    conv1 = Conv2D(48, (3,3), strides=(2,2), activation='relu', padding='same', kernel_initializer=initializers.glorot_uniform(seed = setseed), bias_initializer=initializers.glorot_uniform(seed = setseed))(input)
    mp1 = MaxPooling2D(pool_size=(2, 2), strides=(2,2))(conv1)
    ban1 = BatchNormalization()(mp1)
    
    conv2 = Conv2D(96, (3,3), activation='relu', padding='same', kernel_initializer=initializers.glorot_uniform(seed = setseed), bias_initializer=initializers.glorot_uniform(seed = setseed))(ban1)
    mp2 = MaxPooling2D(pool_size=(3, 3), strides=(2,2))(conv2)
    ban2 = BatchNormalization()(mp2)    
    
    conv3 = Conv2D(192, (3,3), activation='relu', padding='same', kernel_initializer=initializers.glorot_uniform(seed = setseed), bias_initializer=initializers.glorot_uniform(seed = setseed))(ban2)
    conv4 = Conv2D(192, (3,3), activation='relu', padding='same', kernel_initializer=initializers.glorot_uniform(seed = setseed), bias_initializer=initializers.glorot_uniform(seed = setseed))(conv3)
    conv5 = Conv2D(256, (3,3), activation='relu', padding='same', kernel_initializer=initializers.glorot_uniform(seed = setseed), bias_initializer=initializers.glorot_uniform(seed = setseed))(conv4)
    
    mp3 = MaxPooling2D(pool_size=(3, 3), strides=(2,2))(conv5)
    ban3 = BatchNormalization()(mp3) 
    flat= Flatten()(ban3)
    dens1=Dense(512, activation='tanh', kernel_initializer=initializers.glorot_uniform(seed = setseed), bias_initializer=initializers.glorot_uniform(seed = setseed))(flat)
    drop1=Dropout(0.5)(dens1)
    dens2=Dense(256, activation='tanh', kernel_initializer=initializers.glorot_uniform(seed = setseed), bias_initializer=initializers.glorot_uniform(seed = setseed))(drop1)
    drop2=Dropout(0.5)(dens2)   
    probs=Dense(num_classes, activation='softmax', kernel_initializer=initializers.glorot_uniform(seed = setseed), bias_initializer=initializers.glorot_uniform(seed = setseed))(drop2)
    
    model = Model(input=input, output=probs)
    model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics = ['accuracy'])

    return model


# In[4]:


from keras.layers import Input
import keras.backend as K
from keras.models import Model

all_model = [None,None,None]
losses = [None,None,None]

prediction=[]

all_score =[0,0,0]
gr=[]
wr=[]
xwr=[]

for i in range(3):
    np.random.seed(25+i)
    model = build_model(i+2)
    all_model[i]=model
    
for i in range(3):    
    weights = all_model[i].trainable_weights # weight tensors
    weights = [weight for weight in weights] # filter down weights tensors to only ones which are trainable
    gradients = all_model[i].optimizer.get_gradients(all_model[i].total_loss, weights) # gradient tensors
    gr.append(gradients)
    wr.append(weights)
    xweights = all_model[i].non_trainable_weights # weight tensors
    xweights = [weight for weight in xweights] # filter down weights tensors to only ones which are trainable
    xwr.append(xweights)

    losses[i]=all_model[i].total_loss
    prediction.append(all_model[i].output)


# In[5]:


model.summary()


# In[6]:


input_tensors = [all_model[0].inputs[0], # input data
                 all_model[0].sample_weights[0], # how much to weight each sample by
                 all_model[0].targets[0], # labels
                 K.learning_phase(), # train or test mode
                 all_model[1].inputs[0], # input data
                 all_model[1].sample_weights[0], # how much to weight each sample by
                 all_model[1].targets[0], # labels
                 all_model[2].inputs[0], # input data
                 all_model[2].sample_weights[0], # how much to weight each sample by
                 all_model[2].targets[0], # labels
                ]


# In[7]:


minlos = K.argmin(losses)

grr=[]
for x in gr:
    for y in x:
        grr.append(y)

upd_test= K.function(inputs=input_tensors, outputs=[ losses[0], losses[1], losses[2], minlos, prediction[0], prediction[1], prediction[2] ])


# In[8]:


import tensorflow as tf


grad_best=[]
grad_non0 = []
grad_non1 = []


weig_best=[]
weig_non0 = []
weig_non1 = []

xweig_best=[]
xweig_non0 = []
xweig_non1 = []




for i in range(len(gr[0])):
    gr_ck=tf.concat([gr[0][i],gr[1][i], gr[2][i]],0)
    newshape = (3, ) + (tuple(wr[0][i].shape))

    
    gr_ck2=tf.reshape(gr_ck, newshape)
    
    bb = gr_ck2[minlos]
    grad_best.append(bb)
    
    nbb0 = gr_ck2[0:minlos]                       #[0,enk) U (enk,] aralıklarının birleşimi bize nonbesti verecek
    nbb1 = gr_ck2[minlos+1:]                      #[0,enk) U (enk,] aralıklarının birleşimi bize nonbesti verecek
    nbc = tf.concat([nbb0,nbb1], 0)    
    nbc = tf.reshape(nbc, (-1,))
    newshape2 = (2, ) + (tuple(wr[0][i].shape))
    
    nbc2 = tf.reshape(nbc, newshape2) 
    nb0 = nbc2[0]
    nb1 = nbc2[1]
    grad_non0.append(nb0)
    grad_non1.append(nb1)
    

    wr_ck=tf.concat([wr[0][i],wr[1][i], wr[2][i]],0)
    
    newshape = (3, ) + (tuple(wr[0][i].shape))
    wr_ck2=tf.reshape(wr_ck, newshape) 
    bb2 = wr_ck2[minlos]
    weig_best.append(bb2)
    
    #wb = wr_ck[minlos]
    wnbb0 = wr_ck2[0:minlos]                       #[0,enk) U (enk,] aralıklarının birleşimi bize nonbesti verecek
    wnbb1 = wr_ck2[minlos+1:]                      #[0,enk) U (enk,] aralıklarının birleşimi bize nonbesti verecek
    wnbc = tf.concat([wnbb0,wnbb1],0)    
    wnbc = tf.reshape(wnbc, (-1,))
    newshape2 = (2, ) + (tuple(wr[0][i].shape))
    
    wnbc2 =tf.reshape(wnbc, newshape2)
    wnb0 = wnbc2[0]
    wnb1 = wnbc2[1]
    weig_non0.append(wnb0)
    weig_non1.append(wnb1)
    
    if i<len(xwr[0]):
        print (i)
        xwr_ck=tf.concat([xwr[0][i],xwr[1][i], xwr[2][i]], 0)

        newshape = (3, ) + (tuple(xwr[0][i].shape))
        
        xwr_ck2=tf.reshape(xwr_ck, newshape)  
        xbb2 = xwr_ck2[minlos]
        xweig_best.append(xbb2)

        #wb = wr_ck[minlos]
        xwnbb0 = xwr_ck2[0:minlos]                       #[0,enk) U (enk,] aralıklarının birleşimi bize nonbesti verecek
        xwnbb1 = xwr_ck2[minlos+1:]                      #[0,enk) U (enk,] aralıklarının birleşimi bize nonbesti verecek
        xwnbc = tf.concat([xwnbb0,xwnbb1], 0)    
        
        xwnbc = tf.reshape(xwnbc, (-1,))
        newshape2 = (2, ) + (tuple(xwr[0][i].shape))
         
        xwnbc2 = tf.reshape(xwnbc, newshape2) 
        xwnb0 = xwnbc2[0]
        xwnb1 = xwnbc2[1]
        xweig_non0.append(xwnb0)
        xweig_non1.append(xwnb1)
    else:
        pass


# In[9]:


los=tf.stack([losses[0], losses[1], losses[2]])

newshape = (3, )
los2=tf.reshape(los, newshape) 
losbest = los2[minlos]

#wb = wr_ck[minlos]
los_0 = los2[0:minlos]                       #[0,enk) U (enk,] aralıklarının birleşimi bize nonbesti verecek
los_1 = los2[minlos+1:]                      #[0,enk) U (enk,] aralıklarının birleşimi bize nonbesti verecek
loswnbc = tf.concat([los_0,los_1],0)    
loswnbc = tf.reshape(loswnbc,(-1,))
newshape2 = (2, )

loswnbc2 = tf.reshape(loswnbc, newshape2)
losss0 = loswnbc2[0]
losss1 = loswnbc2[1]


Using TensorFlow backend.
C:\Users\ikarabayir\AppData\Local\Continuum\anaconda2\envs\tf\lib\site-packages\ipykernel_launcher.py:85: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 16, 16, 48)        1344      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 8, 8, 48)          0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 8, 8, 48)          192       
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 8, 8, 96)          41568     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 3, 3, 96)          0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 3, 3, 96)          384       
__________

In [22]:

adamb_m = [tf.Variable(tf.zeros(t.shape, dtype=tf.float32, name='m_best')) for t in weig_best]
adamb_v = [tf.Variable(tf.zeros(t.shape, dtype=tf.float32, name='v_best')) for t in weig_best]
adam0_m = [tf.Variable(tf.zeros(t.shape, dtype=tf.float32, name='m_0')) for t in weig_non0]
adam0_v = [tf.Variable(tf.zeros(t.shape, dtype=tf.float32, name='v_0')) for t in weig_non0]
adam1_m = [tf.Variable(tf.zeros(t.shape, dtype=tf.float32, name='m_1')) for t in weig_non1]
adam1_v = [tf.Variable(tf.zeros(t.shape, dtype=tf.float32, name='v_2')) for t in weig_non1]


In [26]:
beta_1 = 0.9
beta_2 = 0.999
step_size = 0.001
eps = 1e-8
t = tf.Variable(1.0, name='iteration')

upd2=[]

for m, v, best, gbest,  param_i, in zip(adamb_m, adamb_v, weig_best, grad_best, wr[2]):
    _m = beta_1 * m + (1 - beta_1) * gbest
    _v = beta_2 * v + (1 - beta_2) * tf.pow(gbest, 2)
    m_hat = _m / (1 - tf.pow(beta_1, t))
    v_hat = _v / (1 - tf.pow(beta_2, t))
    #m_hat = tf.cast(m_hat, tf.float32)
    #v_hat = tf.cast(v_hat, tf.float32)
    #upd2.extend([(m, _m)])
    #upd2.extend([(v, _v)])
    upd2.extend([tf.assign(m, _m)])
    upd2.extend([tf.assign(v, _v)])
    xbest = best - step_size * m_hat / (tf.sqrt(v_hat) + eps)
    #upd2.extend([(param_i, xbest)])
    upd2.extend([tf.assign(param_i, xbest)])


upd2.extend([t.assign_add(1.0)])

upd2.extend([tf.assign(param_i, v)
        for param_i, v in zip(xwr[2], xweig_best)]
    )    
    

In [27]:
upd_bb2= K.function(inputs=input_tensors, outputs=[ losses[0], losses[1], losses[2], minlos, prediction[0], prediction[1], prediction[2] ], updates=upd2)

In [28]:
from keras.preprocessing.image import ImageDataGenerator

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255


In [29]:
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    zca_epsilon=1e-06,  # epsilon for ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    # randomly shift images horizontally (fraction of total width)
    width_shift_range=0.1,
    # randomly shift images vertically (fraction of total height)
    height_shift_range=0.1,
    shear_range=0.,  # set range for random shear
    zoom_range=0.,  # set range for random zoom
    channel_shift_range=0.,  # set range for random channel shifts
    # set mode for filling points outside the input boundaries
    fill_mode='nearest',
    cval=0.,  # value used for fill_mode = "constant"
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False,  # randomly flip images
    # set rescaling factor (applied before any other transformation)
    rescale=None,
    # set function that will be applied on each input
    preprocessing_function=None,
    # image data format, either "channels_first" or "channels_last"
    data_format=None)

# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)


# In[14]:


datagentest = ImageDataGenerator()


# In[15]:


# alfa 0.1 beta 0.5 for nonbest, 0.1 alfa for best
from sklearn.metrics import accuracy_score

lossepoch=[]
lossepoch_test=[]
lossx=[]
acctra=[]
loss_test=[]
acc_test=[]
skip=[]

for f in range(200):
    tr1=[]
    tr2=[]
    res1=[]
    res2=[]
    print('Epoch', f)
    print ('train')
    batches = 0
    for x_batch, y_batch in datagen.flow(x_train, y_train, batch_size=batch_size):
        K.set_learning_phase(1)
        for i in range(len(all_model)):
            all_model[i].layers[14].rate= 0.5
            all_model[i].layers[16].rate= 0.5
        inputs = [x_batch, # X
                  np.ones(y_batch.shape[0]), # sample weights
                  y_batch, # y
                  1, # learning phase in TEST mode
                  x_batch, # X
                  np.ones(y_batch.shape[0]), # sample weights
                  y_batch, # y
                  x_batch, # X
                  np.ones(y_batch.shape[0]), # sample weights
                  y_batch, # y
                 ]
        ll = upd_bb2(inputs)
        yhat=ll[6]
        lossepoch.append(ll[2])
        tr1.append(ll[2])
        tr2.append(accuracy_score(np.argmax(y_batch,axis=1), np.argmax(yhat,axis=1)))
        skip.append(ll[3])
        batches += 1
        if batches > len(x_train) / batch_size:
            # we need to break the loop by hand because
            # the generator loops indefinitely
            break
    m=(len(x_train) / batch_size)-int((len(x_train) / batch_size))
    tr1[-1]*=m
    tr2[-1]*=m
    lossx.append(np.mean(tr1))
    acctra.append(np.mean(tr2))
    print ('train loss score is :'+str(np.mean(tr1)))
    print ('train acc score is :'+str(np.mean(tr2)))
    print ('test')
    batchesx = 0
    for x_batch, y_batch in datagentest.flow(x_test, y_test, batch_size=batch_size):
        K.set_learning_phase(0)
        for i in range(len(all_model)):
            all_model[i].layers[14].rate= 0
            all_model[i].layers[16].rate= 0
        inputs = [x_batch, # X
                  np.ones(y_batch.shape[0]), # sample weights
                  y_batch, # y
                  1, # learning phase in TEST mode
                  x_batch, # X
                  np.ones(y_batch.shape[0]), # sample weights
                  y_batch, # y
                  x_batch, # X
                  np.ones(y_batch.shape[0]), # sample weights
                  y_batch, # y
                 ]
        ll = upd_test(inputs)
        yhat=ll[6]
        lossepoch_test.append(ll[2])
        res1.append(ll[2])
        res2.append(accuracy_score(np.argmax(y_batch,axis=1), np.argmax(yhat,axis=1)))
        batchesx += 1
        if batchesx >= len(x_test) / batch_size:
            break
    m=(len(x_test) / batch_size)-int((len(x_test) / batch_size))
    res1[-1]*=m
    res2[-1]*=m
    loss_test.append(np.mean(res1))
    acc_test.append(np.mean(res2))
    print ('test loss score is :'+str(np.mean(res1)))
    print ('test acc score is :'+str(np.mean(res2)))


Epoch 0
train
train loss score is :1.641719488117396
train acc score is :0.42844868925831203
test
test loss score is :1.2766623463057265
test acc score is :0.5346123417721519
Epoch 1
train
train loss score is :1.2207874412960409
train acc score is :0.5723905051150895
test
test loss score is :1.0571034959034076
test acc score is :0.6252966772151899
Epoch 2
train
train loss score is :1.0870582391234005
train acc score is :0.6222826086956522
test
test loss score is :0.9883694147007375
test acc score is :0.6438884493670886
Epoch 3
train
train loss score is :0.9907171572451396
train acc score is :0.6607057225063938
test
test loss score is :0.9352454394102097
test acc score is :0.666435917721519
Epoch 4
train
train loss score is :0.9271227505505847
train acc score is :0.6831042199488491
test
test loss score is :0.873243958512439
test acc score is :0.689873417721519
Epoch 5
train
train loss score is :0.8816479871340115
train acc score is :0.7005674552429667
test
test loss score is :0.83852850

train loss score is :0.37399929229293943
train acc score is :0.8734414961636828
test
test loss score is :0.6660121041291123
test acc score is :0.787381329113924
Epoch 48
train
train loss score is :0.36716907349465144
train acc score is :0.8759590792838875
test
test loss score is :0.6661314421062228
test acc score is :0.7908425632911392
Epoch 49
train
train loss score is :0.3594636295343299
train acc score is :0.8794157608695652
test
test loss score is :0.6790520107444329
test acc score is :0.7877768987341772
Epoch 50
train
train loss score is :0.35940375979370476
train acc score is :0.8802149936061381
test
test loss score is :0.6918040232945092
test acc score is :0.7884691455696202
Epoch 51
train
train loss score is :0.3592611526703591
train acc score is :0.8783567774936062
test
test loss score is :0.6757061394709575
test acc score is :0.791435917721519
Epoch 52
train
train loss score is :0.34568424776310813
train acc score is :0.8828924232736572
test
test loss score is :0.679666915932

train loss score is :0.23645506360951593
train acc score is :0.9205562659846548
test
test loss score is :0.7456963370683827
test acc score is :0.7906447784810127
Epoch 95
train
train loss score is :0.22780835189287316
train acc score is :0.9229339833759591
test
test loss score is :0.7656797910038429
test acc score is :0.7850079113924051
Epoch 96
train
train loss score is :0.23402653500209075
train acc score is :0.921875
test
test loss score is :0.7397399369297148
test acc score is :0.7884691455696202
Epoch 97
train
train loss score is :0.23341474477249338
train acc score is :0.9209159207161125
test
test loss score is :0.7637062619758558
test acc score is :0.7882713607594937
Epoch 98
train
train loss score is :0.2322481539138519
train acc score is :0.9217950767263428
test
test loss score is :0.7589647887816912
test acc score is :0.784315664556962
Epoch 99
train
train loss score is :0.22716388096818535
train acc score is :0.9235134271099744
test
test loss score is :0.7474272448711002
tes

test loss score is :0.7942795245141923
test acc score is :0.7861946202531646
Epoch 141
train
train loss score is :0.17997751759408076
train acc score is :0.9381993286445013
test
test loss score is :0.8019487408897544
test acc score is :0.7901503164556962
Epoch 142
train
train loss score is :0.181668031884505
train acc score is :0.9404571611253197
test
test loss score is :0.7906853384986708
test acc score is :0.7913370253164557
Epoch 143
train
train loss score is :0.17526601222069824
train acc score is :0.9401374680306905
test
test loss score is :0.7991308943380283
test acc score is :0.7931170886075949
Epoch 144
train
train loss score is :0.17578990237258585
train acc score is :0.9412963554987213
test
test loss score is :0.8220266129208517
test acc score is :0.7871835443037974
Epoch 145
train
train loss score is :0.18221335461758592
train acc score is :0.9400175831202046
test
test loss score is :0.7833029978165899
test acc score is :0.791435917721519
Epoch 146
train
train loss score is 

train loss score is :0.1472400861942326
train acc score is :0.9513067455242967
test
test loss score is :0.82794852215278
test acc score is :0.7909414556962026
Epoch 188
train
train loss score is :0.14939313587468223
train acc score is :0.9498081841432225
test
test loss score is :0.8484632020132451
test acc score is :0.7908425632911392
Epoch 189
train
train loss score is :0.1468553999605615
train acc score is :0.9507472826086957
test
test loss score is :0.8302010759428332
test acc score is :0.789754746835443
Epoch 190
train
train loss score is :0.15147916501021141
train acc score is :0.9493086636828645
test
test loss score is :0.8384543538470811
test acc score is :0.7932159810126582
Epoch 191
train
train loss score is :0.1503507299467807
train acc score is :0.9490489130434783
test
test loss score is :0.8183758806295788
test acc score is :0.7899525316455697
Epoch 192
train
train loss score is :0.14812170419022633
train acc score is :0.9507273017902813
test
test loss score is :0.843996196

In [30]:
np.savetxt("Adam_lossepoch.csv", lossepoch, delimiter=",", fmt='%s')
np.savetxt("Adam_lossepoch_test.csv", lossepoch_test, delimiter=",", fmt='%s')
np.savetxt("Adam_loss_tra.csv", lossx, delimiter=",", fmt='%s')
np.savetxt("Adam_skip.csv", skip, delimiter=",", fmt='%s')
np.savetxt("Adam_acc_tra.csv", acctra, delimiter=",", fmt='%s')
np.savetxt("Adam_loss_test.csv", loss_test, delimiter=",", fmt='%s')
np.savetxt("Adam_acc_test.csv", acc_test, delimiter=",", fmt='%s')

In [31]:
lossx

[1.641719488117396,
 1.2207874412960409,
 1.0870582391234005,
 0.9907171572451396,
 0.9271227505505847,
 0.8816479871340115,
 0.8415727243017967,
 0.8059592935854517,
 0.7785042186679743,
 0.7504598098566465,
 0.735496882720829,
 0.7032908971428566,
 0.6880906758939519,
 0.6676658841273974,
 0.6401928688787744,
 0.6388818841532368,
 0.6170150778444526,
 0.6056280717482347,
 0.5933442163231123,
 0.5740721527763339,
 0.5683817454539907,
 0.5539049276381808,
 0.5480554912934827,
 0.5318290164403598,
 0.5251559565020034,
 0.5172608327454008,
 0.5078341996540194,
 0.49770000213971527,
 0.48937131249157667,
 0.48095689885451665,
 0.46822806092365016,
 0.4655147509273056,
 0.4581878584573793,
 0.4563413394991394,
 0.44629513420869626,
 0.4386080483169964,
 0.4295955555289603,
 0.4291493937068278,
 0.42031252566162886,
 0.410697263553548,
 0.40847515494889003,
 0.40545050681704453,
 0.3977651999467779,
 0.3944968752505834,
 0.38484500879255096,
 0.3805092279525364,
 0.3812742710990064,
 0.3739

In [32]:
acctra

[0.42844868925831203,
 0.5723905051150895,
 0.6222826086956522,
 0.6607057225063938,
 0.6831042199488491,
 0.7005674552429667,
 0.7148937020460358,
 0.72818094629156,
 0.7354739450127877,
 0.7472026854219949,
 0.7521779092071611,
 0.7605099104859335,
 0.7692615089514067,
 0.7768342391304348,
 0.7838075447570333,
 0.7850263746803069,
 0.792219469309463,
 0.7946571291560103,
 0.7986333120204604,
 0.8072450447570333,
 0.8092231457800512,
 0.8136588874680307,
 0.8146978900255755,
 0.8222506393861893,
 0.8208320012787724,
 0.8265864769820972,
 0.8299432544757033,
 0.8337396099744245,
 0.8361772698209718,
 0.8385549872122762,
 0.8415720907928389,
 0.8434303069053708,
 0.844309462915601,
 0.8453085038363172,
 0.850263746803069,
 0.8519621163682864,
 0.8538802749360613,
 0.8550191815856778,
 0.8572570332480819,
 0.8623321611253197,
 0.863031489769821,
 0.863590952685422,
 0.8651694373401535,
 0.8657488810741688,
 0.8706441815856778,
 0.871463395140665,
 0.8717830882352942,
 0.8734414961636828,

In [33]:
loss_test

[1.2766623463057265,
 1.0571034959034076,
 0.9883694147007375,
 0.9352454394102097,
 0.873243958512439,
 0.8385285061748722,
 0.8119883676872978,
 0.8049572643977178,
 0.7795245270185833,
 0.7647297640767279,
 0.7321994197142275,
 0.7086206377495693,
 0.715099998478648,
 0.7075017805718169,
 0.7313109849449955,
 0.7051068155825893,
 0.692116311645206,
 0.6818106755425658,
 0.7009423072390919,
 0.6814848095933094,
 0.6652104606545424,
 0.6726734614636325,
 0.6494818803253053,
 0.6672363333121131,
 0.6590828142022784,
 0.6486156831813764,
 0.6530917890464203,
 0.6782885663494279,
 0.6566752602782431,
 0.65690671284742,
 0.6631265879620479,
 0.6679428792263888,
 0.6737789783296706,
 0.6602060240280779,
 0.6795235332808917,
 0.6662027281296404,
 0.6689705282826967,
 0.654323241213643,
 0.6729629630152183,
 0.6667796199363244,
 0.6683753928806209,
 0.6568412196032608,
 0.673943053035042,
 0.6575209800201126,
 0.6729878636100625,
 0.6654063350037683,
 0.6741555119051209,
 0.6660121041291123,

In [34]:
acc_test

[0.5346123417721519,
 0.6252966772151899,
 0.6438884493670886,
 0.666435917721519,
 0.689873417721519,
 0.7035205696202531,
 0.7146954113924051,
 0.7169699367088608,
 0.725870253164557,
 0.7317049050632911,
 0.7474287974683544,
 0.7524723101265823,
 0.7537579113924051,
 0.7539556962025317,
 0.7467365506329114,
 0.756131329113924,
 0.7611748417721519,
 0.7667128164556962,
 0.7631526898734177,
 0.7676028481012658,
 0.7730419303797469,
 0.7731408227848101,
 0.7745253164556962,
 0.7768987341772152,
 0.778184335443038,
 0.7783821202531646,
 0.7797666139240507,
 0.7743275316455697,
 0.7806566455696202,
 0.7808544303797469,
 0.7818433544303798,
 0.7777887658227848,
 0.7761075949367089,
 0.7780854430379747,
 0.7787776898734177,
 0.7804588607594937,
 0.7822389240506329,
 0.7857990506329114,
 0.7852056962025317,
 0.7813488924050633,
 0.7837223101265823,
 0.7876780063291139,
 0.7827333860759493,
 0.786689082278481,
 0.7861946202531646,
 0.7865901898734177,
 0.7847112341772152,
 0.787381329113924,

In [35]:
np.max(acc_test), np.argmax(acc_test)

(0.7967761075949367, 195)

In [36]:
np.min(loss_test), np.argmin(loss_test)

(0.6486156831813764, 25)